In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,Input

In [2]:
from tensorflow.keras.models import Model
import numpy as np

In [3]:
path = './dataset.xlsx'
df = pd.read_excel(path)
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [4]:
stats = df.describe()
print(stats)

               X1          X2          X3          X4         X5          X6  \
count  768.000000  768.000000  768.000000  768.000000  768.00000  768.000000   
mean     0.764167  671.708333  318.500000  176.604167    5.25000    3.500000   
std      0.105777   88.086116   43.626481   45.165950    1.75114    1.118763   
min      0.620000  514.500000  245.000000  110.250000    3.50000    2.000000   
25%      0.682500  606.375000  294.000000  140.875000    3.50000    2.750000   
50%      0.750000  673.750000  318.500000  183.750000    5.25000    3.500000   
75%      0.830000  741.125000  343.000000  220.500000    7.00000    4.250000   
max      0.980000  808.500000  416.500000  220.500000    7.00000    5.000000   

               X7         X8          Y1          Y2  
count  768.000000  768.00000  768.000000  768.000000  
mean     0.234375    2.81250   22.307195   24.587760  
std      0.133221    1.55096   10.090204    9.513306  
min      0.000000    0.00000    6.010000   10.900000  
25% 

In [5]:
stats.pop('Y1')
stats.pop('Y2')
t_stats = stats.transpose()
t_stats

,count,mean,std,min,25%,50%,75%,max
X1,768.0,0.764167,0.105777,0.62,0.6825,0.75,0.830,0.98
X2,768.0,671.708333,88.086116,514.50,606.3750,673.75,741.125,808.50
X3,768.0,318.500000,43.626481,245.00,294.0000,318.50,343.000,416.50
X4,768.0,176.604167,45.165950,110.25,140.8750,183.75,220.500,220.50
X5,768.0,5.250000,1.751140,3.50,3.5000,5.25,7.000,7.00
X6,768.0,3.500000,1.118763,2.00,2.7500,3.50,4.250,5.00
X7,768.0,0.234375,0.133221,0.00,0.1000,0.25,0.400,0.40
X8,768.0,2.812500,1.550960,0.00,1.7500,3.00,4.000,5.00


In [6]:
def get_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1,y2

In [7]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(df,test_size=0.2)

In [8]:
train_labels = get_output(train)
test_labels = get_output(test)

In [9]:
def normalization(x):
    return (x-t_stats['mean'])/t_stats['std']

In [10]:
x_train = normalization(train)
x_test = normalization(test)

In [11]:
x_train.head()

,X1,X2,X3,X4,X5,X6,X7,X8
631,1.284142,-1.228438,0.000000,-1.197897,0.999349,1.340767,1.243239,0.120893
411,-0.228467,0.162247,-1.684757,0.971879,-0.999349,1.340767,0.117287,0.120893
447,0.527838,-0.672164,0.000000,-0.655453,0.999349,1.340767,0.117287,0.765655
81,-0.701157,0.718520,-0.561586,0.971879,-0.999349,-0.446922,-1.008666,-1.168631
93,-1.362924,1.552931,1.123171,0.971879,-0.999349,-0.446922,-1.008666,-1.168631


In [12]:
train.head()

,X1,X2,X3,X4,X5,X6,X7,X8
631,0.90,563.5,318.5,122.5,7.0,5,0.40,3
411,0.74,686.0,245.0,220.5,3.5,5,0.25,3
447,0.82,612.5,318.5,147.0,7.0,5,0.25,4
81,0.69,735.0,294.0,220.5,3.5,3,0.10,1
93,0.62,808.5,367.5,220.5,3.5,3,0.10,1


In [13]:
input_layer = Input(shape=(8))
f_dense = Dense(128,activation='relu')(input_layer)
s_dense = Dense(64,activation='relu')(f_dense)
out1_layer = Dense(1,name='first_output')(s_dense)

fo_dense = Dense(32,activation='relu')(s_dense)
out2_layer = Dense(1,name='second_output')(fo_dense)

In [14]:
model = Model(inputs=input_layer,outputs=[out1_layer,out2_layer])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          1152        input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 64)           8256        dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 32)           2080        dense_1[0][0]                    
_______________________________________________________________________________________

In [15]:
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.001),
             loss={'first_output':'mse','second_output':'mse'},
             metrics={'first_output':tf.keras.metrics.RootMeanSquaredError(),
                     'second_output':tf.keras.metrics.RootMeanSquaredError()})

In [16]:
model.fit(x_train,train_labels,epochs=500,batch_size=20,validation_data=(x_test,test_labels))

Epoch 1/500
31/31 [==============================] - 0s 9ms/step - loss: 398.7983 - first_output_loss: 196.6939 - second_output_loss: 202.1044 - first_output_root_mean_squared_error: 14.0248 - second_output_root_mean_squared_error: 14.2163 - val_loss: 41.7060 - val_first_output_loss: 18.1532 - val_second_output_loss: 23.5528 - val_first_output_root_mean_squared_error: 4.2607 - val_second_output_root_mean_squared_error: 4.8531
Epoch 2/500
31/31 [==============================] - 0s 1ms/step - loss: 40.3633 - first_output_loss: 17.5501 - second_output_loss: 22.8131 - first_output_root_mean_squared_error: 4.1893 - second_output_root_mean_squared_error: 4.7763 - val_loss: 26.5102 - val_first_output_loss: 11.2012 - val_second_output_loss: 15.3090 - val_first_output_root_mean_squared_error: 3.3468 - val_second_output_root_mean_squared_error: 3.9127
Epoch 3/500
31/31 [==============================] - 0s 1ms/step - loss: 32.1789 - first_output_loss: 13.1369 - second_output_loss: 19.0420 - fir

31/31 [==============================] - 0s 1ms/step - loss: 15.8686 - first_output_loss: 6.3990 - second_output_loss: 9.4695 - first_output_root_mean_squared_error: 2.5296 - second_output_root_mean_squared_error: 3.0773 - val_loss: 17.1773 - val_first_output_loss: 6.7067 - val_second_output_loss: 10.4706 - val_first_output_root_mean_squared_error: 2.5897 - val_second_output_root_mean_squared_error: 3.2358
Epoch 21/500
31/31 [==============================] - 0s 1ms/step - loss: 14.6406 - first_output_loss: 6.1441 - second_output_loss: 8.4964 - first_output_root_mean_squared_error: 2.4787 - second_output_root_mean_squared_error: 2.9149 - val_loss: 21.1300 - val_first_output_loss: 7.6896 - val_second_output_loss: 13.4405 - val_first_output_root_mean_squared_error: 2.7730 - val_second_output_root_mean_squared_error: 3.6661
Epoch 22/500
31/31 [==============================] - 0s 2ms/step - loss: 15.2985 - first_output_loss: 6.3125 - second_output_loss: 8.9861 - first_output_root_mean_squ

31/31 [==============================] - 0s 2ms/step - loss: 11.3083 - first_output_loss: 4.5571 - second_output_loss: 6.7512 - first_output_root_mean_squared_error: 2.1347 - second_output_root_mean_squared_error: 2.5983 - val_loss: 12.6981 - val_first_output_loss: 4.8600 - val_second_output_loss: 7.8382 - val_first_output_root_mean_squared_error: 2.2045 - val_second_output_root_mean_squared_error: 2.7997
Epoch 40/500
31/31 [==============================] - 0s 1ms/step - loss: 10.9152 - first_output_loss: 4.4020 - second_output_loss: 6.5132 - first_output_root_mean_squared_error: 2.0981 - second_output_root_mean_squared_error: 2.5521 - val_loss: 12.6840 - val_first_output_loss: 5.0094 - val_second_output_loss: 7.6745 - val_first_output_root_mean_squared_error: 2.2382 - val_second_output_root_mean_squared_error: 2.7703
Epoch 41/500
31/31 [==============================] - 0s 1ms/step - loss: 10.9390 - first_output_loss: 4.1657 - second_output_loss: 6.7733 - first_output_root_mean_squar

31/31 [==============================] - 0s 3ms/step - loss: 7.4182 - first_output_loss: 2.8381 - second_output_loss: 4.5801 - first_output_root_mean_squared_error: 1.6847 - second_output_root_mean_squared_error: 2.1401 - val_loss: 13.3958 - val_first_output_loss: 5.0457 - val_second_output_loss: 8.3501 - val_first_output_root_mean_squared_error: 2.2463 - val_second_output_root_mean_squared_error: 2.8897
Epoch 59/500
31/31 [==============================] - 0s 4ms/step - loss: 7.8818 - first_output_loss: 2.6061 - second_output_loss: 5.2757 - first_output_root_mean_squared_error: 1.6143 - second_output_root_mean_squared_error: 2.2969 - val_loss: 8.8067 - val_first_output_loss: 3.4597 - val_second_output_loss: 5.3470 - val_first_output_root_mean_squared_error: 1.8600 - val_second_output_root_mean_squared_error: 2.3124
Epoch 60/500
31/31 [==============================] - 0s 4ms/step - loss: 8.0487 - first_output_loss: 2.9524 - second_output_loss: 5.0963 - first_output_root_mean_squared_e

31/31 [==============================] - 0s 1ms/step - loss: 6.7493 - first_output_loss: 2.2259 - second_output_loss: 4.5234 - first_output_root_mean_squared_error: 1.4919 - second_output_root_mean_squared_error: 2.1268 - val_loss: 7.8274 - val_first_output_loss: 2.8761 - val_second_output_loss: 4.9513 - val_first_output_root_mean_squared_error: 1.6959 - val_second_output_root_mean_squared_error: 2.2252
Epoch 78/500
31/31 [==============================] - 0s 1ms/step - loss: 5.7289 - first_output_loss: 1.9925 - second_output_loss: 3.7363 - first_output_root_mean_squared_error: 1.4116 - second_output_root_mean_squared_error: 1.9330 - val_loss: 7.1201 - val_first_output_loss: 2.9785 - val_second_output_loss: 4.1416 - val_first_output_root_mean_squared_error: 1.7258 - val_second_output_root_mean_squared_error: 2.0351
Epoch 79/500
31/31 [==============================] - 0s 2ms/step - loss: 6.6398 - first_output_loss: 2.2963 - second_output_loss: 4.3436 - first_output_root_mean_squared_er

...

31/31 [==============================] - 0s 1ms/step - loss: 1.4830 - first_output_loss: 0.3817 - second_output_loss: 1.1013 - first_output_root_mean_squared_error: 0.6178 - second_output_root_mean_squared_error: 1.0494 - val_loss: 4.9423 - val_first_output_loss: 0.7904 - val_second_output_loss: 4.1519 - val_first_output_root_mean_squared_error: 0.8890 - val_second_output_root_mean_squared_error: 2.0376
Epoch 420/500
31/31 [==============================] - 0s 1ms/step - loss: 1.0211 - first_output_loss: 0.2583 - second_output_loss: 0.7628 - first_output_root_mean_squared_error: 0.5082 - second_output_root_mean_squared_error: 0.8734 - val_loss: 1.2718 - val_first_output_loss: 0.2654 - val_second_output_loss: 1.0065 - val_first_output_root_mean_squared_error: 0.5151 - val_second_output_root_mean_squared_error: 1.0032
Epoch 421/500
31/31 [==============================] - 0s 1ms/step - loss: 1.2715 - first_output_loss: 0.3506 - second_output_loss: 0.9209 - first_output_root_mean_squared_

31/31 [==============================] - 0s 1ms/step - loss: 1.4055 - first_output_loss: 0.3308 - second_output_loss: 1.0748 - first_output_root_mean_squared_error: 0.5751 - second_output_root_mean_squared_error: 1.0367 - val_loss: 1.1716 - val_first_output_loss: 0.2648 - val_second_output_loss: 0.9069 - val_first_output_root_mean_squared_error: 0.5145 - val_second_output_root_mean_squared_error: 0.9523
Epoch 439/500
31/31 [==============================] - 0s 1ms/step - loss: 1.1681 - first_output_loss: 0.3003 - second_output_loss: 0.8678 - first_output_root_mean_squared_error: 0.5480 - second_output_root_mean_squared_error: 0.9316 - val_loss: 1.1611 - val_first_output_loss: 0.2272 - val_second_output_loss: 0.9339 - val_first_output_root_mean_squared_error: 0.4767 - val_second_output_root_mean_squared_error: 0.9664
Epoch 440/500
31/31 [==============================] - 0s 1ms/step - loss: 0.9707 - first_output_loss: 0.2278 - second_output_loss: 0.7429 - first_output_root_mean_squared_

31/31 [==============================] - 0s 1ms/step - loss: 0.7838 - first_output_loss: 0.2196 - second_output_loss: 0.5642 - first_output_root_mean_squared_error: 0.4686 - second_output_root_mean_squared_error: 0.7511 - val_loss: 1.0315 - val_first_output_loss: 0.2279 - val_second_output_loss: 0.8036 - val_first_output_root_mean_squared_error: 0.4774 - val_second_output_root_mean_squared_error: 0.8964
Epoch 458/500
31/31 [==============================] - 0s 1ms/step - loss: 0.6625 - first_output_loss: 0.1700 - second_output_loss: 0.4925 - first_output_root_mean_squared_error: 0.4123 - second_output_root_mean_squared_error: 0.7018 - val_loss: 1.0463 - val_first_output_loss: 0.2346 - val_second_output_loss: 0.8117 - val_first_output_root_mean_squared_error: 0.4844 - val_second_output_root_mean_squared_error: 0.9009
Epoch 459/500
31/31 [==============================] - 0s 1ms/step - loss: 1.2944 - first_output_loss: 0.2977 - second_output_loss: 0.9967 - first_output_root_mean_squared_

31/31 [==============================] - 0s 1ms/step - loss: 0.6060 - first_output_loss: 0.1579 - second_output_loss: 0.4481 - first_output_root_mean_squared_error: 0.3974 - second_output_root_mean_squared_error: 0.6694 - val_loss: 1.0144 - val_first_output_loss: 0.2318 - val_second_output_loss: 0.7826 - val_first_output_root_mean_squared_error: 0.4815 - val_second_output_root_mean_squared_error: 0.8846
Epoch 477/500
31/31 [==============================] - 0s 1ms/step - loss: 1.1692 - first_output_loss: 0.3102 - second_output_loss: 0.8590 - first_output_root_mean_squared_error: 0.5570 - second_output_root_mean_squared_error: 0.9268 - val_loss: 1.3440 - val_first_output_loss: 0.2538 - val_second_output_loss: 1.0902 - val_first_output_root_mean_squared_error: 0.5038 - val_second_output_root_mean_squared_error: 1.0441
Epoch 478/500
31/31 [==============================] - 0s 1ms/step - loss: 1.4275 - first_output_loss: 0.4227 - second_output_loss: 1.0048 - first_output_root_mean_squared_

31/31 [==============================] - 0s 2ms/step - loss: 1.1803 - first_output_loss: 0.2893 - second_output_loss: 0.8910 - first_output_root_mean_squared_error: 0.5378 - second_output_root_mean_squared_error: 0.9439 - val_loss: 1.1023 - val_first_output_loss: 0.2586 - val_second_output_loss: 0.8436 - val_first_output_root_mean_squared_error: 0.5085 - val_second_output_root_mean_squared_error: 0.9185
Epoch 496/500
31/31 [==============================] - 0s 3ms/step - loss: 0.9000 - first_output_loss: 0.1968 - second_output_loss: 0.7032 - first_output_root_mean_squared_error: 0.4437 - second_output_root_mean_squared_error: 0.8385 - val_loss: 0.9941 - val_first_output_loss: 0.2233 - val_second_output_loss: 0.7708 - val_first_output_root_mean_squared_error: 0.4726 - val_second_output_root_mean_squared_error: 0.8780
Epoch 497/500
31/31 [==============================] - 0s 2ms/step - loss: 0.8669 - first_output_loss: 0.1870 - second_output_loss: 0.6799 - first_output_root_mean_squared_